# Installing dependencies

In [1]:
!pip install transformers datasets -q

# Loading dataset

In [2]:
from datasets import load_dataset

ds = load_dataset("satpalsr/chatml-translation-filter", split = 'validation')

In [3]:
ds

Dataset({
    features: ['conversations'],
    num_rows: 324
})

In [4]:
# Sample conversation
ds['conversations'][0]

[{'from': 'system',
  'value': 'For a given question assess whether translating the potential answer to another language might yield an inaccurate response. Avoid translation in tasks related to coding problems, alliteration, idioms, paraphrasing text, word count, spelling correction, and other linguistic constructs or contextual nuances that may affect the accuracy of the answer. When translation is deemed unsuitable, output {"translate": False}. Otherwise, output {"translate": True}.'},
 {'from': 'human',
  'value': 'Can you suggest 10 types of plants that are commonly used in traditional medicine?'},
 {'from': 'gpt', 'value': '{"translate": True}'}]

# Inferencing

In [5]:
from transformers import AutoTokenizer
import transformers
import torch

model = "satpalsr/llama2-translation-filter-full"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-06-23 13:16:53.031315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 13:16:53.031369: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 13:16:53.033002: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 13:16:53.040166: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use avail

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
sample_input = '''
<|im_start|>system
For a given question assess whether translating the potential answer to another language might yield an inaccurate response. Avoid translation in tasks related to coding problems, alliteration, idioms, paraphrasing text, word count, spelling correction, and other linguistic constructs or contextual nuances that may affect the accuracy of the answer. When translation is deemed unsuitable, output {"translate": False}. Otherwise, output {"translate": True}.<|im_end|>
<|im_start|>user
Generate a paragraph based on the given information. In the year 2025, the United Nations is working hard to reduce its carbon emissions by 50%.<|im_end|>
<|im_start|>assistant
'''

In [7]:
# Sample from the dataset
ds['conversations'][0]

[{'from': 'system',
  'value': 'For a given question assess whether translating the potential answer to another language might yield an inaccurate response. Avoid translation in tasks related to coding problems, alliteration, idioms, paraphrasing text, word count, spelling correction, and other linguistic constructs or contextual nuances that may affect the accuracy of the answer. When translation is deemed unsuitable, output {"translate": False}. Otherwise, output {"translate": True}.'},
 {'from': 'human',
  'value': 'Can you suggest 10 types of plants that are commonly used in traditional medicine?'},
 {'from': 'gpt', 'value': '{"translate": True}'}]

In [9]:
validation_set = []
for convo in ds['conversations']:
    sample = {}
    template = f'''<|im_start|>system
{convo[0]['value']}<|im_end|>
<|im_start|>user
{convo[1]['value']}<|im_end|>
<|im_start|>assistant
'''
    sample['input'] = template
    sample['true_value'] = convo[2]['value']
    sample['output'] = ""
    validation_set.append(sample)

In [10]:
validation_set[0]

{'input': '<|im_start|>system\nFor a given question assess whether translating the potential answer to another language might yield an inaccurate response. Avoid translation in tasks related to coding problems, alliteration, idioms, paraphrasing text, word count, spelling correction, and other linguistic constructs or contextual nuances that may affect the accuracy of the answer. When translation is deemed unsuitable, output {"translate": False}. Otherwise, output {"translate": True}.<|im_end|>\n<|im_start|>user\nCan you suggest 10 types of plants that are commonly used in traditional medicine?<|im_end|>\n<|im_start|>assistant\n',
 'true_value': '{"translate": True}',
 'output': ''}

In [18]:
i =0
for sample in validation_set:
    sequences = pipeline(
        sample['input'],
        max_new_tokens = 100,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        # max_length=500,
    )
    sample['output'] = sequences[0]['generated_text']
    print(i)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
validation_set[0]['output'], validation_set[0]['true_value']

('<|im_start|>system\nFor a given question assess whether translating the potential answer to another language might yield an inaccurate response. Avoid translation in tasks related to coding problems, alliteration, idioms, paraphrasing text, word count, spelling correction, and other linguistic constructs or contextual nuances that may affect the accuracy of the answer. When translation is deemed unsuitable, output {"translate": False}. Otherwise, output {"translate": True}.<|im_end|>\n<|im_start|>user\nCan you suggest 10 types of plants that are commonly used in traditional medicine?<|im_end|>\n<|im_start|>assistant\n{"translate": False}<|im_end|><|im_end|>\n',
 '{"translate": True}')

In [20]:
validation_set[3]['true_value'] , validation_set[3]['output'].split('<|im_start|>')[-1]

('{"translate": False}', 'assistant\n{"translate": False}<|im_end|>\n')

In [21]:
s = validation_set[3]['output'].split('<|im_start|>')[-1]
(validation_set[3]['true_value'] in s.split('<|im_end|>')[0])

True

In [36]:
# Accuracy when model is trained for 200 steps 
correct = 0
for i in range(0,len(validation_set)):
        s = validation_set[i]['output'].split('assistant')[-1]
        if(validation_set[i]['true_value'] in s.split('<|im_end|>')[0]):
            correct +=1

In [37]:
print(f"Accuracy of the model is {(correct/len(validation_set))*100}%")

Accuracy of the model is 59.876543209876544%


In [34]:
# Accuracy when model is trained for 200 steps 
correct = 0
for i in range(0,len(validation_set)):
        s = validation_set[i]['output'].split('assistant')[-1]
        if(validation_set[i]['true_value'] in s):
            correct +=1

In [35]:
print(f"Accuracy of the model is {(correct/len(validation_set))*100}%")

Accuracy of the model is 63.888888888888886%


In [26]:
positive = 0
for i in range(0,len(validation_set)):
    if("True" in validation_set[i]['true_value']):
        positive +=1

In [27]:
print((positive/len(validation_set))*100)

47.22222222222222


In [22]:
for i in range(0,len(validation_set)):
    print(validation_set[i]['output'].split('<|im_start|>')[-1])
    print("===========")

assistant
{"translate": False}<|im_end|><|im_end|>

assistant

{"translate": False}
assistant
{"translate": False}<|im_end|>

assistant
{"translate": False}<|im_end|>

assistant
{"translate": False}<|im_end|>
assistant
{"translate": False}<|im_end|>

assistant
{"translate": True}<|im_end|>

assistant
{"translate": False}<|im_end|> 

assistant
{"translate": False}<|im_end|> 

assistant
{"translate": False}<|im_end|>

assistant
{"translate": False}<|im_end|>
{"translate": False}<|im_end|>

assistant
{"translate": False}
assistant
{"translate": False}<|im_end|><|im_end|> 

assistant
{"translate": False}<|im_end|> 

assistant
{"translate": False}<|im_end|>

assistant
{"translate": True}<|im_end|><|im_end|><|im_end|>
assistant
{"translate": False}<|im_end|>

{"translate": False}<|im_end|>
{"translate": False}
{"translate": False}
{"translate": False}

assistant
{"translate": False}<|im_end|>

assistant

assistant
{"translate": False}<|im_end|>

assistant
{"translate": False}<|im_end|>


ass